<a href="https://colab.research.google.com/github/peppopi/Piratas/blob/main/Target_66_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@markdown ### Inicie el Drive si lo necesita
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import shutil
import sqlite3
import hashlib
import imghdr
#@markdown ### Crea el Datastet
#@markdown ###  Ruta de la carpeta con los archivos .txt y las imágenes (Formato Standar)
dataset_folder = '/content/drive/MyDrive/lora_training/datasets/ampharos' #@param {type:"string"}

# Ruta de la carpeta del dataset
output_folder = 'my_dataset'

# Crear la carpeta del dataset si no existe
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Crear la carpeta 'images' dentro del dataset si no existe
images_folder = os.path.join(output_folder, 'images')
if not os.path.exists(images_folder):
    os.makedirs(images_folder)

# Conectar a la base de datos
conn = sqlite3.connect(os.path.join(output_folder, 'my-dataset.sqlite'))

# Crear la tabla 'posts'
conn.execute('''
    CREATE TABLE IF NOT EXISTS posts (
        id INTEGER PRIMARY KEY,
        md5 TEXT,
        file_ext TEXT,
        tag_string TEXT,
        tag_count_general INTEGER
    )
''')

# Recorrer los archivos .txt y las imágenes en la carpeta
for filename in os.listdir(dataset_folder):
    if filename.endswith('.txt'):
        # Leer el archivo .txt
        with open(os.path.join(dataset_folder, filename), 'r') as file:
            # Obtener el nombre de la imagen a partir del nombre del archivo .txt
            image_name = os.path.splitext(filename)[0]

            # Leer los tags del archivo .txt
            tag_string = file.readline().strip()

            # Calcular el hash MD5 del nombre de la imagen
            md5 = hashlib.md5(image_name.encode()).hexdigest()

            # Buscar el archivo de imagen en la carpeta
            image_path = None
            for root, dirs, files in os.walk(dataset_folder):
                for file in files:
                    if file.startswith(image_name):
                        image_path = os.path.join(root, file)
                        break
                if image_path:
                    break

            # Asignar la extensión de archivo utilizando imghdr
            image_extension = imghdr.what(image_path)

            # Insertar el registro en la tabla 'posts'
            conn.execute('INSERT INTO posts (id, md5, file_ext, tag_string, tag_count_general) VALUES (?, ?, ?, ?, ?)',
                         (image_name, md5, image_extension, tag_string, len(tag_string.split())))

            # Copiar la imagen a la carpeta 'images' del dataset si se encontró la ruta
            if image_path:
                shutil.copy(image_path, os.path.join(images_folder, f"{image_name}.{image_extension}"))
            else:
                print(f"No se encontró la imagen para {image_name}")


# Cerrar la conexión a la base de datos
conn.commit()
conn.close()



In [ ]:
#@markdown ### Entrena Tu Chekpoint
##Saca Advertencias
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
##
import os

# Instalar las dependencias con pip en modo silencioso
!pip install -qq tensorflow tensorflow-io Click numpy requests scikit-image six

# Descargar el repositorio de DeepDanbooru desde GitHub
!git clone -q https://github.com/KichangKim/DeepDanbooru

# Acceder a la carpeta de DeepDanbooru
%cd DeepDanbooru

# Instalar DeepDanbooru junto con TensorFlow en modo silencioso
!pip install -qq .[tensorflow]

# Crear un proyecto en DeepDanbooru
!deepdanbooru create-project my_project


import requests
import os

# URL del archivo tags.txt
url = 'https://github.com/peppopi/Piratas/raw/main/tags.txt'

# Ruta de destino para guardar el archivo
destination_folder = '/content/DeepDanbooru/my_project'

# Nombre de archivo para guardar el archivo descargado
filename = 'tags.txt'

# Ruta completa del archivo de destino
destination_path = os.path.join(destination_folder, filename)

# Descargar el archivo
response = requests.get(url)
if response.status_code == 200:
    with open(destination_path, 'wb') as file:
        file.write(response.content)
    print('El archivo se ha descargado correctamente.')
else:
    print('No se pudo descargar el archivo.')

import os
import json

# Ruta del proyecto DeepDanbooru
project_folder = '/content/DeepDanbooru/my_project'

# Ruta del archivo SQLite
sqlite_path = '/content/DeepDanbooru/my_project/my-dataset.sqlite'

# Copiar tu dataset a la carpeta del proyecto (reemplaza /ruta/a/tu/dataset por la ruta real de tu dataset)
!mkdir -p {os.path.join(project_folder, 'dataset', 'images')}
!cp -r /content/my_dataset* {os.path.join(project_folder, 'dataset')}

import os

source_folder = '/content/DeepDanbooru/my_project/dataset/my_dataset'
destination_folder = '/content/DeepDanbooru/my_project'

# Mover los archivos y carpetas al destino
!mv {source_folder}/* {destination_folder}

# Eliminar la carpeta de origen vacía
!rm -r {source_folder}

# Modificar el archivo project.json en la carpeta del proyecto
project_json_path = os.path.join(project_folder, 'project.json')
if os.path.exists(project_json_path):
    with open(project_json_path, 'r') as f:
        project_data = json.load(f)
    project_data['database_path'] = sqlite_path
    with open(project_json_path, 'w') as f:
        json.dump(project_data, f, indent=4)

import json

# Ruta del archivo project.json
project_json_path = '/content/DeepDanbooru/my_project/project.json'

# Cargar los datos actuales del archivo project.json
with open(project_json_path, 'r') as f:
    project_data = json.load(f)

# Modificar las características deseadas
#@markdown Establece el ancho de las imágenes que se utilizarán durante el entrenamiento y evaluación del modelo
project_data['image_width'] = 299 #@param {type:"string"}

#@markdown Establece el alto de las imágenes que se utilizarán durante el entrenamiento y evaluación del modelo
project_data['image_height'] = 299 #@param {type:"string"}

# Especifica la ruta del archivo de base de datos SQLite que contiene la información de las imágenes y sus etiquetas
project_data['database_path'] = '/content/DeepDanbooru/my_project/my-dataset.sqlite'

#@markdown Define el recuento mínimo de etiquetas requerido para que una imagen sea considerada en el entrenamiento
project_data['minimum_tag_count'] = 20 #@param {type:"string"}

#@markdown Especifica el modelo que se utilizará para el entrenamiento y evaluación
project_data['model'] = 'resnet_custom_v2'  #@param {type:"string"}

#@markdown Establece el tamaño del minibatch utilizado durante el entrenamiento
project_data['minibatch_size'] = 32 #@param {type:"string"}

#@markdown Define la cantidad de epochs (iteraciones completas a través del conjunto de datos) que se realizarán durante el entrenamiento
project_data['epoch_count'] = 10 #@param {type:"string"}

#@markdown Especifica la frecuencia con la que se exportará el modelo durante el entrenamiento
project_data['export_model_per_epoch'] = 10 #@param {type:"string"}

#@markdown Define la frecuencia con la que se guardará un punto de control (checkpoint) del modelo durante el entrenamiento
project_data['checkpoint_frequency_mb'] = 200 #@param {type:"string"}

#@markdown Establece la frecuencia con la que se mostrarán los registros de progreso en la consola durante el entrenamiento
project_data['console_logging_frequency_mb'] = 10 #@param {type:"string"}

#@markdown Especifica la función de pérdida que se utilizará durante el entrenamiento
project_data['loss'] = 'binary_crossentropy'  #@param {type:"string"}

#@markdown Define el optimizador que se utilizará para ajustar los pesos del modelo durante el entrenamiento
project_data['optimizer'] = 'adam'  #@param {type:"string"}

#@markdown Establece la tasa de aprendizaje del optimizador
project_data['learning_rate'] = 0.001 #@param {type:"string"}

#Define el rango de rotación que se aplicará a las imágenes durante el entrenamiento
project_data['rotation_range'] = [0.0, 360.0]

# Define el rango de escala que se aplicará a las imágenes durante el entrenamiento
project_data['scale_range'] = [0.9, 1.1]

# Define el rango de desplazamiento horizontal y vertical que se aplicará a las imágenes durante el entrenamiento
project_data['shift_range'] = [-0.1, 0.1]

#@markdown Indica si se utilizará la precisión mixta durante el entrenamiento
project_data['mixed_precision'] = False #@param {type:"boolean"}



# Guardar los cambios en el archivo project.json
with open(project_json_path, 'w') as f:
    json.dump(project_data, f, indent=4)


# Entrenar el proyecto
!deepdanbooru train-project {project_folder}

print('La carpeta donde segurdaron tus chekpoints es la siguiente: /content/DeepDanbooru/my_project/checkpoints')



In [ ]:
!deepdanbooru evaluate /content/drive/MyDrive/lora_training/datasets/kythe/0d50e3283f08.jpeg --project-path {project_folder} --allow-folder

In [ ]:
#@markdown Tutorial de Hiperparametros

print("project_data['image_width'] y project_data['image_height']: Establecen el ancho y alto de las imágenes que se utilizarán durante el entrenamiento y evaluación del modelo. Estos valores se establecen en 256 en el ejemplo.")

print("project_data['database_path']: Especifica la ruta del archivo de base de datos SQLite que contiene la información de las imágenes y sus etiquetas. En este caso, se establece como 'my_dataset/new-dataset.sqlite', lo que indica que el archivo de base de datos se encuentra en la carpeta 'my_dataset' y se llama 'new-dataset.sqlite'.")

print("project_data['minimum_tag_count']: Define el recuento mínimo de etiquetas requerido para que una imagen sea considerada en el entrenamiento. Las imágenes que no cumplan con este requisito se ignorarán. En este caso, se establece en 10.")

print("project_data['model']: Especifica el modelo que se utilizará para el entrenamiento y evaluación. En el ejemplo se utiliza el modelo 'resnet_custom_v3'.")

print("project_data['minibatch_size']: Establece el tamaño del minibatch utilizado durante el entrenamiento. Un minibatch es un subconjunto de muestras utilizado para calcular los gradientes y actualizar los pesos del modelo. En este caso, se establece en 64.")

print("project_data['epoch_count']: Define la cantidad de epochs (iteraciones completas a través del conjunto de datos) que se realizarán durante el entrenamiento. En el ejemplo se establece en 20.")

print("project_data['export_model_per_epoch']: Especifica la frecuencia con la que se exportará el modelo durante el entrenamiento. En este caso, se exportará el modelo cada 5 epochs.")

print("project_data['checkpoint_frequency_mb']: Define la frecuencia con la que se guardará un punto de control (checkpoint) del modelo durante el entrenamiento. En el ejemplo, se guarda un punto de control cada 100 minibatches.")

print("project_data['console_logging_frequency_mb']: Establece la frecuencia con la que se mostrarán los registros de progreso en la consola durante el entrenamiento. En este caso, se mostrarán los registros cada 5 minibatches.")

print("project_data['loss']: Especifica la función de pérdida que se utilizará durante el entrenamiento. En el ejemplo se utiliza la pérdida 'categorical_crossentropy', que es comúnmente utilizada en problemas de clasificación.")

print("project_data['optimizer']: Define el optimizador que se utilizará para ajustar los pesos del modelo durante el entrenamiento. En el ejemplo se utiliza el optimizador 'sgd', que representa el descenso de gradiente estocástico.")

print("project_data['learning_rate']: Establece la tasa de aprendizaje del optimizador. Esta tasa determina qué tan rápido se actualizarán los pesos del modelo durante el entrenamiento. En el ejemplo se establece en 0.01.")

print("project_data['rotation_range'], project_data['scale_range'] y project_data['shift_range']: Estos parámetros definen los rangos de rotación, escala y desplazamiento horizontal y vertical aplicados a las imágenes durante el entrenamiento. En el ejemplo se utilizan rangos específicos.")

print("project_data['mixed_precision']: Indica si se utilizará la precisión mixta durante el entrenamiento. La precisión mixta es una técnica que utiliza operaciones en punto flotante de menor precisión para acelerar el entrenamiento. En este caso, se establece en True para utilizar la precisión mixta.")


print('''
{EJEMPLO BASE:
    "image_width": 299,
    "image_height": 299,
    "database_path": "my_dataset/my-dataset.sqlite",
    "minimum_tag_count": 20,
    "model": "resnet_custom_v2",
    "minibatch_size": 32,
    "epoch_count": 10,
    "export_model_per_epoch": 10,
    "checkpoint_frequency_mb": 200,
    "console_logging_frequency_mb": 10,
    "loss": "binary_crossentropy",
    "optimizer": "adam",
    "learning_rate": 0.001,
    "rotation_range": [
        0.0,
        360.0
    ],
    "scale_range": [
        0.9,
        1.1
    ],
    "shift_range": [
        -0.1,
        0.1
    ],
    "mixed_precision": false
}
''')


print("Para mas informacion consultar el paper : https://github.com/KichangKim/DeepDanbooru")
